In [2]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# 1. Read files including `Answer`, `Choice`, `Question`

In [4]:
# Read Files to List
doc_num = 91
question_doc = []
choice_doc = []
ans_doc = []

for i in range(1, doc_num + 1):
    try:
        with open(f'Data/Question/{i}.txt', encoding='utf-8') as f:
            raw_q = f.read()
    except:
        with open(f'Data/Question/{i}.txt', encoding='big5') as f:
            raw_q = f.read()
    try:
        with open(f'Data/Choice/{i}.txt', encoding='utf-8') as f:
            raw_ch = f.read()
    except:
        with open(f'Data/Choice/{i}.txt', encoding='big5') as f:
            raw_ch = f.read()
    try:
        with open(f'Data/Answer/{i}.txt', encoding='utf-8') as f:
            raw_a = f.read()
    except:
        with open(f'Data/Answer/{i}.txt', encoding='big5') as f:
            raw_a = f.read()
    question_doc.append(raw_q)
    choice_doc.append(raw_ch.split('\n'))
    ans_doc.append(list(raw_a))

In [5]:
choice_cols = ['A', 'B', 'C', 'D', 'E', 'F']
choice_df = pd.DataFrame(choice_doc, columns=choice_cols, index=[i + 1 for i in range(len(choice_doc))])
for col in choice_cols:
    choice_df[col] = choice_df[col].str.replace(r'\([A-F]\)', '')

ans_cols = [1, 2, 3, 4, 5]
ans_df = pd.DataFrame(ans_doc, columns=ans_cols, index=[i + 1 for i in range(len(choice_doc))])

display(choice_df.head())
display(ans_df.head())

,A,B,C,D,E,F
1,Nothing is made up.,"History is nonfiction, too.",But your trip through space would be fiction.,You could write a story in which you fly to t...,But fiction isn’t always different from the w...,None
2,"But when I pulled into the driveway, there wa...",Hours later I called my daughter and asked if...,"One time, I even shouted at Derek when he unp...",Derek became part of our life and seemed to f...,"Without any experience in raising pets, my hu...",None
3,Another genre commonly found in Chinese brush...,"However, the subject matters later expanded b...","As a result, they have obtained more natural ...",Its growth has inevitably reflected the chang...,It then gradually developed into two separate...,
4,She got to look at the fine old houses as wel...,Architects make drawings and careful plans of...,The word for what she does is rehabilitation.,Then she went on to study architecture.,That is precisely what Carrie does.,None
5,There are definitely two sides to this issue.,Scores decreased as the family size increased...,An intelligence test was administered to over...,"On the other side are Rutherford and Sewell, ...","Since then, the theory has been elaborated an...",None


,1,2,3,4,5
1,E,D,C,A,B
2,E,D,C,A,B
3,D,B,E,A,C
4,C,E,A,D,B
5,C,B,E,A,D


# 2. BERT (NSP) only considering sentence before option
code ref: https://towardsdatascience.com/bert-for-next-sentence-prediction-466b67f8226f

In [6]:
# pip3 install transformers
# pip3 install torch

from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
from torch.nn.functional import softmax

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
count_NotNextSentence = 0
NotNextSentence = []

for i in range(len(question_doc)): # each document
  question_sentence = re.split('\(1\)|\(2\)|\(3\)|\(4\)|\(5\)', question_doc[i])
  quesion_num = 5
  if (ans_df.iloc[i][5] == None):
    quesion_num = 4
  for j in range(quesion_num): # each question
      for k in range(6): # each option
        if (choice_df.iloc[i][choice_cols[k]] != None):
          inputs = tokenizer(question_sentence[j], choice_df.iloc[i][choice_cols[k]], return_tensors='pt')
          outputs = model(**inputs)
          if (torch.argmax(outputs.logits).item() == 1): # NotNextSentence
            count_NotNextSentence += 1
            NotNextSentence.append([i+1, j+1, choice_cols[k]]) # Doc i+1, Question j+1, Choice choice_cols[k], is not the answer
  print("=== Finish Doc", i+1, "===")

=== Finish Doc 1 ===
=== Finish Doc 2 ===
=== Finish Doc 3 ===
=== Finish Doc 4 ===
=== Finish Doc 5 ===
=== Finish Doc 6 ===
=== Finish Doc 7 ===
=== Finish Doc 8 ===
=== Finish Doc 9 ===
=== Finish Doc 10 ===
=== Finish Doc 11 ===
=== Finish Doc 12 ===
=== Finish Doc 13 ===
=== Finish Doc 14 ===
=== Finish Doc 15 ===
=== Finish Doc 16 ===
=== Finish Doc 17 ===
=== Finish Doc 18 ===
=== Finish Doc 19 ===
=== Finish Doc 20 ===
=== Finish Doc 21 ===
=== Finish Doc 22 ===
=== Finish Doc 23 ===
=== Finish Doc 24 ===
=== Finish Doc 25 ===
=== Finish Doc 26 ===
=== Finish Doc 27 ===
=== Finish Doc 28 ===
=== Finish Doc 29 ===
=== Finish Doc 30 ===
=== Finish Doc 31 ===
=== Finish Doc 32 ===
=== Finish Doc 33 ===
=== Finish Doc 34 ===
=== Finish Doc 35 ===
=== Finish Doc 36 ===
=== Finish Doc 37 ===
=== Finish Doc 38 ===
=== Finish Doc 39 ===
=== Finish Doc 40 ===
=== Finish Doc 41 ===
=== Finish Doc 42 ===
=== Finish Doc 43 ===
=== Finish Doc 44 ===
=== Finish Doc 45 ===
=== Finish Doc 46 =

In [8]:
# In all situations (91*5*5 = 2375), only `count_NotNextSentence` situations are defined as `NotNextSentence`
print(count_NotNextSentence)

188


In [9]:
print(NotNextSentence)

[[3, 4, 'D'], [4, 1, 'D'], [4, 2, 'C'], [4, 2, 'D'], [4, 2, 'E'], [4, 3, 'B'], [5, 3, 'A'], [6, 1, 'B'], [6, 4, 'B'], [8, 5, 'D'], [10, 1, 'A'], [10, 1, 'C'], [10, 1, 'D'], [10, 3, 'C'], [13, 1, 'C'], [13, 2, 'C'], [14, 1, 'C'], [14, 2, 'C'], [14, 3, 'C'], [17, 5, 'F'], [18, 5, 'D'], [19, 1, 'B'], [19, 1, 'C'], [19, 2, 'C'], [19, 3, 'C'], [19, 4, 'B'], [20, 2, 'C'], [20, 3, 'C'], [20, 5, 'C'], [22, 1, 'A'], [22, 1, 'B'], [22, 1, 'D'], [22, 1, 'F'], [22, 2, 'B'], [22, 2, 'F'], [22, 3, 'B'], [22, 3, 'E'], [22, 3, 'F'], [22, 4, 'A'], [22, 4, 'D'], [22, 4, 'E'], [22, 5, 'A'], [22, 5, 'C'], [22, 5, 'E'], [23, 1, 'B'], [27, 4, 'C'], [31, 1, 'B'], [31, 1, 'C'], [31, 3, 'A'], [31, 4, 'A'], [33, 1, 'A'], [33, 2, 'A'], [34, 2, 'A'], [38, 1, 'B'], [38, 3, 'B'], [41, 2, 'B'], [41, 4, 'B'], [43, 1, 'B'], [43, 4, 'B'], [43, 4, 'F'], [44, 1, 'B'], [44, 1, 'C'], [44, 5, 'A'], [45, 1, 'B'], [45, 1, 'D'], [45, 1, 'E'], [45, 2, 'B'], [45, 3, 'B'], [45, 3, 'C'], [45, 4, 'B'], [46, 3, 'E'], [46, 4, 'C'], [

In [10]:
for i in range(len(NotNextSentence)):
  if (ans_df.iloc[NotNextSentence[i][0]-1][NotNextSentence[i][1]] == NotNextSentence[i][2]):
    print("Wrong answer!!", NotNextSentence[i])

Wrong answer!! [4, 2, 'E']
Wrong answer!! [47, 4, 'A']
Wrong answer!! [52, 3, 'F']
Wrong answer!! [53, 4, 'B']
Wrong answer!! [60, 5, 'E']
Wrong answer!! [63, 5, 'C']


# 3. BERT (NSP) consider sentence after option

In [11]:
count_answer = 0
count_not_answer = 0
answer = []
not_answer = []

for i in range(len(question_doc)): # each document
  question_sentence = re.split('\(1\)|\(2\)|\(3\)|\(4\)|\(5\)', question_doc[i])
  quesion_num = 5
  if (ans_df.iloc[i][5] == None):
    quesion_num = 4
  for j in range(quesion_num): # each question
    for k in range(6): # each option
      if (choice_df.iloc[i][choice_cols[k]] != None):
        inputs = tokenizer(question_sentence[j], choice_df.iloc[i][choice_cols[k]], return_tensors='pt')
        outputs = model(**inputs)
        if (torch.argmax(outputs.logits).item() == 0 and len(question_sentence) > j): # IsNextSentence
          inputs_2 = tokenizer(choice_df.iloc[i][choice_cols[k]], question_sentence[j+1], return_tensors='pt')
          outputs_2 = model(**inputs_2)
          if (torch.argmax(outputs_2.logits).item() == 0): # IsNextSentence
            count_answer += 1
            answer.append([i+1, j+1, choice_cols[k]]) # Doc i+1, Question j+1, Choice choice_cols[k], is the answer
          else:
            count_not_answer += 1
            not_answer.append([i+1, j+1, choice_cols[k]])
        elif (torch.argmax(outputs.logits).item() == 0): # IsNextSentence but no more content after the option field
          count_answer += 1
          answer.append([i+1, j+1, choice_cols[k]]) # Doc i+1, Question j+1, Choice choice_cols[k], is the answer
        else:
          count_not_answer += 1
          not_answer.append([i+1, j+1, choice_cols[k]])
  print("=== Finish Doc", i+1, "===")

=== Finish Doc 1 ===
=== Finish Doc 2 ===
=== Finish Doc 3 ===
=== Finish Doc 4 ===
=== Finish Doc 5 ===
=== Finish Doc 6 ===
=== Finish Doc 7 ===
=== Finish Doc 8 ===
=== Finish Doc 9 ===
=== Finish Doc 10 ===
=== Finish Doc 11 ===
=== Finish Doc 12 ===
=== Finish Doc 13 ===
=== Finish Doc 14 ===
=== Finish Doc 15 ===
=== Finish Doc 16 ===
=== Finish Doc 17 ===
=== Finish Doc 18 ===
=== Finish Doc 19 ===
=== Finish Doc 20 ===
=== Finish Doc 21 ===
=== Finish Doc 22 ===
=== Finish Doc 23 ===
=== Finish Doc 24 ===
=== Finish Doc 25 ===
=== Finish Doc 26 ===
=== Finish Doc 27 ===
=== Finish Doc 28 ===
=== Finish Doc 29 ===
=== Finish Doc 30 ===
=== Finish Doc 31 ===
=== Finish Doc 32 ===
=== Finish Doc 33 ===
=== Finish Doc 34 ===
=== Finish Doc 35 ===
=== Finish Doc 36 ===
=== Finish Doc 37 ===
=== Finish Doc 38 ===
=== Finish Doc 39 ===
=== Finish Doc 40 ===
=== Finish Doc 41 ===
=== Finish Doc 42 ===
=== Finish Doc 43 ===
=== Finish Doc 44 ===
=== Finish Doc 45 ===
=== Finish Doc 46 =

In [12]:
len(answer)

2072

In [13]:
len(not_answer)

308

In [14]:
for i in range(len(not_answer)):
  if (ans_df.iloc[not_answer[i][0]-1][not_answer[i][1]] == not_answer[i][2]):
    print("Wrong answer!!", not_answer[i])

Wrong answer!! [4, 2, 'E']
Wrong answer!! [22, 3, 'D']
Wrong answer!! [47, 4, 'A']
Wrong answer!! [50, 5, 'B']
Wrong answer!! [52, 3, 'F']
Wrong answer!! [53, 1, 'E']
Wrong answer!! [53, 4, 'B']
Wrong answer!! [60, 5, 'E']
Wrong answer!! [62, 1, 'C']
Wrong answer!! [63, 5, 'C']
Wrong answer!! [69, 1, 'A']


In [15]:
answer

[[1, 1, 'A'],
 [1, 1, 'B'],
 [1, 1, 'C'],
 [1, 1, 'D'],
 [1, 1, 'E'],
 [1, 2, 'A'],
 [1, 2, 'B'],
 [1, 2, 'C'],
 [1, 2, 'D'],
 [1, 2, 'E'],
 [1, 3, 'A'],
 [1, 3, 'B'],
 [1, 3, 'C'],
 [1, 3, 'D'],
 [1, 3, 'E'],
 [1, 4, 'A'],
 [1, 4, 'B'],
 [1, 4, 'C'],
 [1, 4, 'D'],
 [1, 4, 'E'],
 [1, 5, 'A'],
 [1, 5, 'B'],
 [1, 5, 'C'],
 [1, 5, 'D'],
 [1, 5, 'E'],
 [2, 1, 'A'],
 [2, 1, 'B'],
 [2, 1, 'C'],
 [2, 1, 'D'],
 [2, 1, 'E'],
 [2, 2, 'A'],
 [2, 2, 'B'],
 [2, 2, 'C'],
 [2, 2, 'D'],
 [2, 2, 'E'],
 [2, 3, 'A'],
 [2, 3, 'B'],
 [2, 3, 'C'],
 [2, 3, 'D'],
 [2, 3, 'E'],
 [2, 4, 'A'],
 [2, 4, 'B'],
 [2, 4, 'C'],
 [2, 4, 'D'],
 [2, 4, 'E'],
 [2, 5, 'B'],
 [2, 5, 'C'],
 [2, 5, 'D'],
 [3, 1, 'A'],
 [3, 1, 'B'],
 [3, 1, 'C'],
 [3, 1, 'D'],
 [3, 1, 'E'],
 [3, 1, 'F'],
 [3, 2, 'A'],
 [3, 2, 'B'],
 [3, 2, 'C'],
 [3, 2, 'D'],
 [3, 2, 'E'],
 [3, 3, 'A'],
 [3, 3, 'B'],
 [3, 3, 'C'],
 [3, 3, 'E'],
 [3, 3, 'F'],
 [3, 4, 'A'],
 [3, 4, 'B'],
 [3, 4, 'C'],
 [3, 4, 'E'],
 [3, 4, 'F'],
 [3, 5, 'A'],
 [3, 5, 'B'],
 [3, 5

In [16]:
doc = 1
question = 1
tmp_count = 0
answer_count = []

for i in range(len(answer)):
  if (answer[i][0] == doc and answer[i][1] == question):
    tmp_count += 1
  else:
    answer_count.append([doc, question, tmp_count])
    doc = answer[i][0]
    question = answer[i][1]
    tmp_count = 1

In [17]:
answer_count

[[1, 1, 5],
 [1, 2, 5],
 [1, 3, 5],
 [1, 4, 5],
 [1, 5, 5],
 [2, 1, 5],
 [2, 2, 5],
 [2, 3, 5],
 [2, 4, 5],
 [2, 5, 3],
 [3, 1, 6],
 [3, 2, 5],
 [3, 3, 5],
 [3, 4, 5],
 [3, 5, 5],
 [4, 1, 3],
 [4, 2, 2],
 [4, 3, 4],
 [4, 4, 4],
 [4, 5, 5],
 [5, 1, 4],
 [5, 2, 4],
 [5, 3, 4],
 [5, 4, 5],
 [5, 5, 5],
 [6, 1, 4],
 [6, 2, 5],
 [6, 3, 3],
 [6, 4, 4],
 [6, 5, 5],
 [7, 1, 5],
 [7, 2, 5],
 [7, 3, 5],
 [7, 4, 5],
 [7, 5, 5],
 [8, 1, 5],
 [8, 2, 5],
 [8, 3, 5],
 [8, 4, 4],
 [8, 5, 4],
 [9, 1, 5],
 [9, 2, 5],
 [9, 3, 4],
 [9, 4, 5],
 [9, 5, 4],
 [10, 1, 2],
 [10, 2, 4],
 [10, 3, 4],
 [10, 4, 5],
 [10, 5, 5],
 [11, 1, 5],
 [11, 2, 6],
 [11, 3, 6],
 [11, 4, 6],
 [11, 5, 6],
 [12, 1, 6],
 [12, 2, 6],
 [12, 3, 6],
 [12, 4, 6],
 [12, 5, 6],
 [13, 1, 5],
 [13, 2, 3],
 [13, 3, 5],
 [13, 4, 5],
 [13, 5, 3],
 [14, 1, 5],
 [14, 2, 5],
 [14, 3, 5],
 [14, 4, 6],
 [14, 5, 5],
 [15, 1, 6],
 [15, 2, 6],
 [15, 3, 5],
 [15, 4, 6],
 [15, 5, 6],
 [16, 1, 6],
 [16, 2, 6],
 [16, 3, 6],
 [16, 4, 6],
 [16, 5, 6],
 [17,

In [18]:
answer_count_stat = np.zeros(7)

for i in range(len(answer_count)):
  answer_count_stat[answer_count[i][2]] += 1
  if ((answer_count[i][2]) == 1):
    print(answer_count[i])
    print("correct answer", ans_df.iloc[answer_count[i][0] - 1][answer_count[i][1]])
    for j in range(len(answer)):
      if (answer[j][0] == answer_count[i][0] and answer[j][1] == answer_count[i][1]):
        print("our answer", answer[j][2])
        break

print(answer_count_stat)


[22, 3, 1]
correct answer D
our answer C
[50, 5, 1]
correct answer B
our answer C
[60, 4, 1]
correct answer D
our answer D
[60, 5, 1]
correct answer E
our answer D
[79, 1, 1]
correct answer D
our answer D
[84, 5, 1]
correct answer E
our answer E
[  0.   6.  13.  35. 109. 128. 143.]


# 4. BERT (NSP) using probability and considering both previous and next sentence

In [19]:
answer_prob = []

for i in range(len(question_doc)): # each document
  tmp_answer_prob_for_doc = []
  question_sentence = re.split('\(1\)|\(2\)|\(3\)|\(4\)|\(5\)', question_doc[i])
  quesion_num = 5
  if (ans_df.iloc[i][5] == None):
    quesion_num = 4
  for j in range(quesion_num): # each question
    tmp_answer_prob_for_question = []
    for k in range(6): # each option
      if (choice_df.iloc[i][choice_cols[k]] != None):
        # sentence before
        inputs = tokenizer(question_sentence[j], choice_df.iloc[i][choice_cols[k]], return_tensors='pt')
        outputs = model(**inputs)[0]
        probs = softmax(outputs, dim=1)
        # sentence after
        if (len(question_sentence) > j):
          inputs_2 = tokenizer(choice_df.iloc[i][choice_cols[k]], question_sentence[j+1], return_tensors='pt')
          outputs_2 = model(**inputs_2)[0]
          probs_2 = softmax(outputs_2, dim=1)
          tmp_answer_prob_for_question.append((probs[0][0].item() + probs_2[0][0].item()) / 2)
        else:
          tmp_answer_prob_for_question.append(probs[0][0].item())
    tmp_answer_prob_for_doc.append(tmp_answer_prob_for_question)
  answer_prob.append(tmp_answer_prob_for_doc)

  print("=== Finish Doc", i+1, "===")

=== Finish Doc 1 ===
=== Finish Doc 2 ===
=== Finish Doc 3 ===
=== Finish Doc 4 ===
=== Finish Doc 5 ===
=== Finish Doc 6 ===
=== Finish Doc 7 ===
=== Finish Doc 8 ===
=== Finish Doc 9 ===
=== Finish Doc 10 ===
=== Finish Doc 11 ===
=== Finish Doc 12 ===
=== Finish Doc 13 ===
=== Finish Doc 14 ===
=== Finish Doc 15 ===
=== Finish Doc 16 ===
=== Finish Doc 17 ===
=== Finish Doc 18 ===
=== Finish Doc 19 ===
=== Finish Doc 20 ===
=== Finish Doc 21 ===
=== Finish Doc 22 ===
=== Finish Doc 23 ===
=== Finish Doc 24 ===
=== Finish Doc 25 ===
=== Finish Doc 26 ===
=== Finish Doc 27 ===
=== Finish Doc 28 ===
=== Finish Doc 29 ===
=== Finish Doc 30 ===
=== Finish Doc 31 ===
=== Finish Doc 32 ===
=== Finish Doc 33 ===
=== Finish Doc 34 ===
=== Finish Doc 35 ===
=== Finish Doc 36 ===
=== Finish Doc 37 ===
=== Finish Doc 38 ===
=== Finish Doc 39 ===
=== Finish Doc 40 ===
=== Finish Doc 41 ===
=== Finish Doc 42 ===
=== Finish Doc 43 ===
=== Finish Doc 44 ===
=== Finish Doc 45 ===
=== Finish Doc 46 =

In [20]:
print(answer_prob)

[[[0.9999937415122986, 0.9999955296516418, 0.9999942779541016, 0.9999942779541016, 0.9999964237213135], [0.9999821186065674, 0.9999749660491943, 0.9999955296516418, 0.9999958872795105, 0.9999942779541016], [0.9999307990074158, 0.9995757341384888, 0.9999966621398926, 0.9999962449073792, 0.999994158744812], [0.9999839663505554, 0.999997079372406, 0.9999945759773254, 0.9999933838844299, 0.9999958276748657], [0.9999681115150452, 0.999996542930603, 0.9999926090240479, 0.9999942779541016, 0.9999950528144836]], [[0.9999740719795227, 0.9999750852584839, 0.9997751414775848, 0.9999536871910095, 0.999996542930603], [0.999686062335968, 0.9999597072601318, 0.999991774559021, 0.9999958276748657, 0.9999949336051941], [0.9999715089797974, 0.9999947547912598, 0.9999942779541016, 0.9999932050704956, 0.9999934434890747], [0.999986469745636, 0.9999943375587463, 0.9999912977218628, 0.9999883770942688, 0.9999826550483704], [0.5006035520345904, 0.9999906420707703, 0.9986215531826019, 0.9998765885829926, 0.50

## 4-1 Choose from highest probability
Start from choosing the highest probability among all questions & options

In [49]:
our_answer_1 = []

for i in range(len(answer_prob)):
  num_q = len(answer_prob[i])
  num_o = len(answer_prob[i][0])
  chosen = np.zeros((num_q, num_o)) # chosen[j][k] = question j, option k
  tmp_answer = np.zeros(num_q)
  for x in range(num_q): # we need to find answers for all questions
    # find the highest probability for this round
    pos_q = 0 # record the highest probability position
    pos_o = 0
    highest_prob = 0
    for j in range(num_q):
      for k in range(num_o):
        if (answer_prob[i][j][k] > highest_prob and chosen[j][k] == 0):
          highest_prob = answer_prob[i][j][k]
          pos_q = j
          pos_o = k
    # record the answer
    tmp_answer[pos_q] = pos_o
    for j in range(num_q):
      chosen[j][pos_o] = 1
    for k in range(num_o):
      chosen[pos_q][k] = 1
  our_answer_1.append(tmp_answer)

In [52]:
print(our_answer_1)

[array([4., 3., 2., 1., 0.]), array([4., 3., 1., 2., 0.]), array([0., 1., 4., 5., 2.]), array([1., 2., 4., 0., 3.]), array([2., 1., 4., 3., 0.]), array([3., 1., 2., 0., 4.]), array([2., 3., 1., 4., 0.]), array([2., 4., 0., 3., 1.]), array([1., 4., 3., 2., 0.]), array([2., 0., 3., 4., 1.]), array([5., 1., 0., 2., 3.]), array([0., 1., 3., 5., 4.]), array([4., 3., 1., 5., 2.]), array([3., 1., 5., 0., 4.]), array([4., 5., 1., 0., 3.]), array([2., 4., 0., 3., 1.]), array([2., 4., 0., 3., 1.]), array([5., 3., 2., 4., 0.]), array([5., 0., 4., 2., 3.]), array([3., 5., 4., 1., 0.]), array([1., 2., 0., 5., 4.]), array([4., 2., 3., 1., 5.]), array([1., 0., 2., 3.]), array([1., 3., 0., 2.]), array([2., 3., 1., 0.]), array([1., 3., 0., 2.]), array([2., 3., 0., 1.]), array([1., 3., 2., 0.]), array([3., 1., 2., 0.]), array([1., 3., 0., 2.]), array([0., 2., 1., 3.]), array([1., 2., 0., 3.]), array([0., 1., 2., 3.]), array([1., 3., 2., 0.]), array([0., 1., 3., 2.]), array([3., 0., 1., 2.]), array([0., 

### Compare our answer with the correct answer

In [60]:
correct_rate = []

for i in range(len(our_answer_1)):
  count = 0
  ours = []
  correct = []
  for j in range(len(our_answer_1[i])):
    correct.append(ans_df.iloc[i][j+1])
    if (our_answer_1[i][j] == 0):
      ours.append('A')
    elif (our_answer_1[i][j] == 1):
      ours.append('B')
    elif (our_answer_1[i][j] == 2):
      ours.append('C')
    elif (our_answer_1[i][j] == 3):
      ours.append('D')
    elif (our_answer_1[i][j] == 4):
      ours.append('E')
    elif (our_answer_1[i][j] == 5):
      ours.append('F')
    if (ans_df.iloc[i][j+1] == ours[j]):
      count += 1
  print("Doc", i+1, ":", count/len(our_answer_1[i]), ours, correct)
  correct_rate.append(count/len(our_answer_1[i]))

Doc 1 : 0.6 ['E', 'D', 'C', 'B', 'A'] ['E', 'D', 'C', 'A', 'B']
Doc 2 : 0.4 ['E', 'D', 'B', 'C', 'A'] ['E', 'D', 'C', 'A', 'B']
Doc 3 : 0.6 ['A', 'B', 'E', 'F', 'C'] ['D', 'B', 'E', 'A', 'C']
Doc 4 : 0.0 ['B', 'C', 'E', 'A', 'D'] ['C', 'E', 'A', 'D', 'B']
Doc 5 : 0.6 ['C', 'B', 'E', 'D', 'A'] ['C', 'B', 'E', 'A', 'D']
Doc 6 : 0.0 ['D', 'B', 'C', 'A', 'E'] ['C', 'E', 'A', 'D', 'B']
Doc 7 : 0.4 ['C', 'D', 'B', 'E', 'A'] ['B', 'D', 'A', 'E', 'C']
Doc 8 : 0.4 ['C', 'E', 'A', 'D', 'B'] ['A', 'E', 'D', 'C', 'B']
Doc 9 : 0.2 ['B', 'E', 'D', 'C', 'A'] ['E', 'C', 'D', 'A', 'B']
Doc 10 : 0.0 ['C', 'A', 'D', 'E', 'B'] ['E', 'D', 'A', 'B', 'C']
Doc 11 : 0.4 ['F', 'B', 'A', 'C', 'D'] ['F', 'D', 'A', 'E', 'C']
Doc 12 : 0.2 ['A', 'B', 'D', 'F', 'E'] ['B', 'A', 'D', 'E', 'F']
Doc 13 : 0.2 ['E', 'D', 'B', 'F', 'C'] ['B', 'D', 'F', 'A', 'E']
Doc 14 : 0.2 ['D', 'B', 'F', 'A', 'E'] ['E', 'A', 'F', 'C', 'D']
Doc 15 : 0.4 ['E', 'F', 'B', 'A', 'D'] ['E', 'A', 'B', 'F', 'C']
Doc 16 : 0.2 ['C', 'E', 'A', 'D', 

In [61]:
print(correct_rate)

[0.6, 0.4, 0.6, 0.0, 0.6, 0.0, 0.4, 0.4, 0.2, 0.0, 0.4, 0.2, 0.2, 0.2, 0.4, 0.2, 0.4, 0.8, 0.2, 0.0, 0.6, 0.8, 0.5, 1.0, 0.0, 0.25, 1.0, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 1.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 1.0, 0.8, 0.6, 0.4, 0.2, 0.2, 0.6, 0.6, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.0, 0.0, 0.4, 0.4, 0.6, 0.2, 0.6, 0.0, 0.4, 0.0, 0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.4, 0.0, 0.2, 0.4, 0.4, 0.2, 0.6, 0.2, 0.6, 1.0, 0.6, 0.2, 0.2, 0.4, 0.6, 0.6, 0.6, 0.2, 0.4, 0.6]


In [64]:
np.mean(correct_rate)

0.37527472527472533

In [65]:
import collections
collections.Counter(correct_rate)

Counter({0.6: 16, 0.4: 16, 0.0: 13, 0.2: 25, 0.8: 3, 0.5: 10, 1.0: 5, 0.25: 3})

In [71]:
print("1. ", np.mean(correct_rate[0:22]), collections.Counter(correct_rate[0:22]))
print("2. ", np.mean(correct_rate[22:42]), collections.Counter(correct_rate[22:42]))
print("3. ", np.mean(correct_rate[42:62]), collections.Counter(correct_rate[42:62]))
print("4. ", np.mean(correct_rate[62:91]), collections.Counter(correct_rate[62:91]))

1.  0.34545454545454546 Counter({0.4: 6, 0.2: 6, 0.6: 4, 0.0: 4, 0.8: 2})
2.  0.4875 Counter({0.5: 10, 1.0: 4, 0.0: 3, 0.25: 3})
3.  0.35 Counter({0.2: 8, 0.6: 5, 0.4: 4, 0.0: 2, 0.8: 1})
4.  0.3379310344827586 Counter({0.2: 11, 0.6: 7, 0.4: 6, 0.0: 4, 1.0: 1})


## 4-2 Use linear sum assignment

In [36]:
from scipy.optimize import linear_sum_assignment
our_answer_2 = []
tmp_prob_for_LSA = answer_prob

for i in range(len(answer_prob)):
  for j in range(len(answer_prob[i])):
    for k in range(len(answer_prob[i][j])):
      tmp_prob_for_LSA[i][j][k] = -tmp_prob_for_LSA[i][j][k]
  row_ind, col_ind = linear_sum_assignment(tmp_prob_for_LSA[i])
  our_answer_2.append(col_ind)
  print(i, col_ind)


0 [0 3 2 4 1]
1 [4 3 2 0 1]
2 [3 1 4 0 2]
3 [0 1 4 2 3]
4 [2 1 4 3 0]
5 [3 4 2 0 1]
6 [1 3 4 2 0]
7 [0 4 3 2 1]
8 [2 4 1 3 0]
9 [4 0 3 1 2]
10 [5 1 0 2 3]
11 [1 0 3 5 4]
12 [0 3 5 2 4]
13 [3 1 0 5 4]
14 [4 1 0 5 3]
15 [5 4 2 3 0]
16 [1 5 0 2 4]
17 [5 3 2 4 0]
18 [5 3 4 0 2]
19 [4 1 5 3 0]
20 [2 1 0 5 4]
21 [4 3 2 1 5]
22 [2 0 1 3]
23 [1 3 0 2]
24 [3 0 1 2]
25 [0 3 1 2]
26 [2 0 3 1]
27 [1 2 0 3]
28 [3 1 2 0]
29 [1 0 3 2]
30 [0 2 1 3]
31 [2 1 0 3]
32 [2 1 0 3]
33 [1 3 2 0]
34 [1 3 0 2]
35 [0 3 1 2]
36 [0 3 1 2]
37 [2 1 3 0]
38 [3 0 1 2]
39 [3 2 1 0]
40 [1 2 3 0]
41 [0 2 3 1]
42 [4 5 0 2 1]
43 [5 0 4 1 2]
44 [2 3 0 4 5]
45 [4 1 5 3 0]
46 [0 4 3 5 2]
47 [5 2 0 1 3]
48 [1 0 2 4 5]
49 [3 4 5 0 2]
50 [1 3 2 5 4]
51 [5 2 0 4 1]
52 [0 3 2 5 1]
53 [1 2 4 0 3]
54 [3 1 5 4 0]
55 [4 0 1 2 5]
56 [3 1 0 2 5]
57 [2 0 4 5 3]
58 [1 2 5 4 0]
59 [2 5 1 3 4]
60 [1 4 0 5 2]
61 [5 4 3 0 1]
62 [5 1 3 4 0]
63 [4 0 3 5 2]
64 [2 3 4 1 5]
65 [2 5 4 1 3]
66 [2 0 5 1 3]
67 [0 1 3 2 5]
68 [5 4 3 2 1]
69 [0 4 2 3 5]


### Compare our answer with the correct answer

In [37]:
correct_rate = []

for i in range(len(our_answer_2)):
  count = 0
  ours = []
  correct = []
  for j in range(len(our_answer_2[i])):
    correct.append(ans_df.iloc[i][j+1])
    if (our_answer_2[i][j] == 0):
      ours.append('A')
    elif (our_answer_2[i][j] == 1):
      ours.append('B')
    elif (our_answer_2[i][j] == 2):
      ours.append('C')
    elif (our_answer_2[i][j] == 3):
      ours.append('D')
    elif (our_answer_2[i][j] == 4):
      ours.append('E')
    elif (our_answer_2[i][j] == 5):
      ours.append('F')
    if (ans_df.iloc[i][j+1] == ours[j]):
      count += 1
  print("Doc", i+1, ":", count/len(our_answer_2[i]), ours, correct)
  correct_rate.append(count/len(our_answer_2[i]))

Doc 1 : 0.6 ['A', 'D', 'C', 'E', 'B'] ['E', 'D', 'C', 'A', 'B']
Doc 2 : 1.0 ['E', 'D', 'C', 'A', 'B'] ['E', 'D', 'C', 'A', 'B']
Doc 3 : 1.0 ['D', 'B', 'E', 'A', 'C'] ['D', 'B', 'E', 'A', 'C']
Doc 4 : 0.0 ['A', 'B', 'E', 'C', 'D'] ['C', 'E', 'A', 'D', 'B']
Doc 5 : 0.6 ['C', 'B', 'E', 'D', 'A'] ['C', 'B', 'E', 'A', 'D']
Doc 6 : 0.4 ['D', 'E', 'C', 'A', 'B'] ['C', 'E', 'A', 'D', 'B']
Doc 7 : 0.4 ['B', 'D', 'E', 'C', 'A'] ['B', 'D', 'A', 'E', 'C']
Doc 8 : 1.0 ['A', 'E', 'D', 'C', 'B'] ['A', 'E', 'D', 'C', 'B']
Doc 9 : 0.0 ['C', 'E', 'B', 'D', 'A'] ['E', 'C', 'D', 'A', 'B']
Doc 10 : 0.6 ['E', 'A', 'D', 'B', 'C'] ['E', 'D', 'A', 'B', 'C']
Doc 11 : 0.4 ['F', 'B', 'A', 'C', 'D'] ['F', 'D', 'A', 'E', 'C']
Doc 12 : 0.6 ['B', 'A', 'D', 'F', 'E'] ['B', 'A', 'D', 'E', 'F']
Doc 13 : 0.6 ['A', 'D', 'F', 'C', 'E'] ['B', 'D', 'F', 'A', 'E']
Doc 14 : 0.0 ['D', 'B', 'A', 'F', 'E'] ['E', 'A', 'F', 'C', 'D']
Doc 15 : 0.4 ['E', 'B', 'A', 'F', 'D'] ['E', 'A', 'B', 'F', 'C']
Doc 16 : 0.6 ['F', 'E', 'C', 'D', 

In [38]:
np.mean(correct_rate)

0.5241758241758242

In [40]:
import collections
collections.Counter(correct_rate)

Counter({0.6: 21, 1.0: 16, 0.0: 6, 0.4: 18, 0.8: 6, 0.2: 13, 0.5: 7, 0.25: 4})

In [41]:
print("1. ", np.mean(correct_rate[0:22]), collections.Counter(correct_rate[0:22]))
print("2. ", np.mean(correct_rate[22:42]), collections.Counter(correct_rate[22:42]))
print("3. ", np.mean(correct_rate[42:62]), collections.Counter(correct_rate[42:62]))
print("4. ", np.mean(correct_rate[62:91]), collections.Counter(correct_rate[62:91]))

1.  0.5272727272727272 Counter({0.6: 7, 0.4: 6, 1.0: 4, 0.0: 3, 0.8: 1, 0.2: 1})
2.  0.675 Counter({1.0: 9, 0.5: 7, 0.25: 4})
3.  0.4800000000000001 Counter({0.2: 5, 0.4: 5, 0.6: 4, 0.8: 4, 0.0: 1, 1.0: 1})
4.  0.44827586206896547 Counter({0.6: 10, 0.4: 7, 0.2: 7, 0.0: 2, 1.0: 2, 0.8: 1})
